In [21]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import os


os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
torch.cuda.empty_cache()
print(torch.cuda.is_available())


# Load the pre-trained Sentence Transformer model
# mdl_name = 'stella_en_1.5B_v5' # oom
# mdl_name = 'stella_en_400M_v5'
# mdl_name = 'all-mpnet-base-v2'
mdl_name = 'all-MiniLM-L6-v2'

# model = SentenceTransformer("dunzhang/"+mdl_name, trust_remote_code=True).cuda()
model = SentenceTransformer(mdl_name, trust_remote_code=True).cuda()

# Function to generate embeddings
def get_embeddings(text):
    # Generate embeddings
    embedding = model.encode(text)
    return embedding

True


In [22]:
# get the fea_df
fea_df = pd.read_csv("../../data/fea_df.csv")
fea_df['description'] = fea_df['description'].apply(
    lambda x: re.sub(r"Must related to [^(]*\(?.*?\)?\.", "", x).strip()
)

In [23]:
refer_str = '' 
returned_str_ls = fea_df.description.tolist()
if mdl_name in ['stella_en_1.5B_v5', 'stella_en_400M_v5']:
    query_embeddings = model.encode([refer_str], prompt_name="s2p_query")
else:
    query_embeddings = model.encode([refer_str])
doc_embeddings = model.encode(returned_str_ls)
cos_scores = cosine_similarity(query_embeddings, doc_embeddings)[0]
fea_df["baseline_cosine"] = cos_scores
fea_df

,fea,feature,description,baseline_cosine
0,relation,Relationships,Family and social relationships,0.118812
1,protein,High protein diet,"High protein diet, carbohydrate-reduced(low-ca...",0.036156
2,ed,ED recovery,"Eating disorders(ED) diagnosis or recovery, ED...",0.089720
3,exercise,Physical exercise,Physical exercise,0.108038
4,meal,Meals,Routine of meals,0.000049
5,crave,Food cravings,Craving for high calorie food or carbs,0.044027
6,restrict,Restriction,Restrict nutrition or calorie intake,-0.045722
7,binge,Binge eating,Binge eating,0.211423
8,loss,Weight loss,Body weight loss,0.114953
9,gain,Weight gain,Body weight gain,0.066228


In [24]:
fea_df.to_csv("../../data/fea_df_with_cos"+mdl_name+".csv")